Importing data generator and training

In [1]:
import sys, os
sys.path.append(os.getcwd())
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import data_gen1

np.random.seed(69)
# torch.device('cuda' if torch.cuda.is_available else 'cpu')
device = torch.device('cpu')

In [2]:
def create_data(num_points, point_size=1000, min_var0=0.1, max_var0=10, min_var1=0.1, max_var1=10, dim=1):
    x0_var = np.random.uniform(min_var0, max_var0, num_points)
    x1_var = np.random.uniform(min_var1, max_var1, num_points)
    data = np.zeros([num_points, point_size, 2, dim])
    label = np.zeros([num_points])

    for i in range(num_points):
        data[i], label[i] = data_gen1.gen_list(
            point_size, x0_var[i], x1_var[i], dim)

    return data, label

def create_data2(realisations, xy_len=1000, min_var0=0.1, max_var0=10, min_var1=0.1, max_var1=10, dim=1):
    x0_var = np.random.uniform(min_var0, max_var0, realisations)
    x1_var = np.random.uniform(min_var1, max_var1, realisations)
    data = np.zeros((realisations, xy_len, 2))
    label = np.zeros([realisations])

    for i in range(realisations):
        _test, label[i] = data_gen1.gen_list(
            xy_len, x0_var[i], x1_var[i], dim)
        x, y = _test[:, 0, :], _test[:, 1, :]

        data[i] = np.hstack((x, y))

    return data, label

data, label = create_data2(2, 2)
print(data, label)


[[[-2.78204166 -0.50442394]
  [ 0.69264511  2.68776457]]

 [[ 2.42784544  1.7955729 ]
  [ 1.96322533  0.49412433]]] [0.44381637 0.50881422]


In [3]:
# Test data import
class Article_nn(nn.Module):
    def __init__(self, input_size):
        super(Article_nn, self).__init__()
        self.l1 = nn.Linear(input_size*2, 64)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(64, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.sig(self.l2(out))
        return out


class NumbersDataset(Dataset):
    def __init__(self, num_points, point_size):
        self.samples, self.labels = create_data2(num_points, point_size)
        self.samples = torch.from_numpy(self.samples).to(torch.float32)
        self.labels = torch.from_numpy(self.labels).to(torch.float32)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]


In [4]:
# training loop
def train(train_loader, learning_rate, num_epoch, input_size):
    model = Article_nn(input_size)
    # print(train_loader)
    # loss and optimizer
    criterion = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    # training loop
    n_total_steps = len(train_loader)
    for epoch in range(num_epoch):
        for i, (images, labels) in enumerate(train_loader):
            sample = images.reshape(5,-1).to(device)
            labels = labels.view(labels.shape[0], 1).to(device) # makes it a column vector

            # forward
            output = model(sample)
            loss = criterion(output, labels)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i) % 1 == 0:
                print(
                    f'epoch {epoch} / {num_epoch-1}, step {i}/{n_total_steps-1} loss = {loss.item():.4f}')

    print(f"\n#################################\n# TEST DONE\n#################################\n")

    return model

In [5]:
def test(model, test_loader):
    with torch.no_grad():
        n_samples = 0
        n_diff = 0
        for images, labels in test_loader:
            sample = images.reshape(5, -1).to(device)
            labels = labels.view(labels.shape[0], 1).to(device)

            outputs = model(sample) # trained model
            n_diff += torch.mean(torch.abs(outputs-labels))
            n_samples += labels.shape[0]
            #n_correct += (pred == labels).sum().item()

        acc = n_diff/n_samples
        print(f"accuracy = {acc}")

In [7]:
if __name__ == '__main__':
    xy_len = 100
    batch_size = 5
    realisations = 20
    train_data = NumbersDataset(realisations, xy_len)
    test_data = NumbersDataset(realisations, xy_len)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

    # train
    model_trained = train(train_loader, 0.5, 20, xy_len)

    # test
    test(model_trained, test_loader)

epoch 0 / 19, step 0/3 loss = 0.8007
epoch 0 / 19, step 1/3 loss = 0.9169
epoch 0 / 19, step 2/3 loss = 0.9257
epoch 0 / 19, step 3/3 loss = 2.9313
epoch 1 / 19, step 0/3 loss = 46.6680
epoch 1 / 19, step 1/3 loss = 6.2802
epoch 1 / 19, step 2/3 loss = -1.8519
epoch 1 / 19, step 3/3 loss = -9.7304
epoch 2 / 19, step 0/3 loss = 7.6825
epoch 2 / 19, step 1/3 loss = 19.7181
epoch 2 / 19, step 2/3 loss = 21.4233
epoch 2 / 19, step 3/3 loss = 20.2777
epoch 3 / 19, step 0/3 loss = 59.3292
epoch 3 / 19, step 1/3 loss = 31.3401
epoch 3 / 19, step 2/3 loss = -1.2533
epoch 3 / 19, step 3/3 loss = 33.1630
epoch 4 / 19, step 0/3 loss = 37.2920
epoch 4 / 19, step 1/3 loss = 36.7015
epoch 4 / 19, step 2/3 loss = 5.0255
epoch 4 / 19, step 3/3 loss = 41.8599
epoch 5 / 19, step 0/3 loss = 27.0690
epoch 5 / 19, step 1/3 loss = 18.5378
epoch 5 / 19, step 2/3 loss = 47.0295
epoch 5 / 19, step 3/3 loss = 28.2411
epoch 6 / 19, step 0/3 loss = 31.2588
epoch 6 / 19, step 1/3 loss = 4.2425
epoch 6 / 19, step 2